In [1]:
# Importação de libs "Padrão"
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta  
from tqdm import tqdm
import shutil
import workdays as wds
from filecmp import cmp
import openpyxl
from filecmp import dircmp
from tqdm import tqdm
import xlrd
import win32com.client
pd.set_option('display.max_rows', None)

# lib para ignorar warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
pin = r'C:\POWERBI\Perdas\Bases\\'
pout = r'C:\POWERBI\Perdas\StarSchema\\'
pval = r'C:\POWERBI\Perdas\Valid\\'
 
list_path = [pin, pout, pval]
 
for path in list_path:
    if not os.path.exists(path):
        os.makedirs(path)
 

In [3]:
# Diretório para captura das bases necessárias
diretorio = r'C:\\POWERBI\\Perdas\\Bases'
nome_arquivo = 'PENDENCIAS CONCILIACAO (DEME) 21 02 2025.xlsx'
nome_arquivo2 = 'P16-Back_3_202501.csv'
diretorio_cambio = r'C:\\POWERBI\\CambioPronto\\Bases\\Base_Consolidada'
nome_arquivo3 = 'hq_online_offline.csv'

In [4]:
# Definir o diretório e o nome do arquivo
caminho_arquivo = os.path.join(diretorio, nome_arquivo)

# Verificar se o arquivo existe
if os.path.exists(caminho_arquivo):
    # Carregar o arquivo Excel em um DataFrame, pulando as primeiras 4 linhas
    df = pd.read_excel(caminho_arquivo, engine='openpyxl', header=4)
    print("Arquivo Excel carregado com sucesso.")
else:
    print("Arquivo Excel não encontrado.")
    exit()

Arquivo Excel carregado com sucesso.


In [5]:
# Deletar as colunas especificadas, se existirem
colunas_para_deletar = ["Produto", "Cuenta", "Área", "Superintendência", "superintendência         ", "superintendência", "Referencia_3"]
colunas_existentes = [col for col in colunas_para_deletar if col in df.columns]
df.drop(columns=colunas_existentes, inplace=True)

# Filtrar as linhas onde a coluna "Justificativa" inicia com "DashPerdas"
if 'Justificativa' in df.columns:
    df = df[df['Justificativa'].str.startswith('DashPerdas', na=False)]
else:
    print("A coluna 'Justificativa' não existe no DataFrame.")
    exit()

In [6]:
# Normatizar todas as colunas removendo espaços em branco e substituindo por underscores
df.columns = df.columns.str.strip().str.replace(' ', '_')

In [7]:
# Definir a nova ordem das colunas
new_order = [
    'Referencia_1', 'Referencia_2', 'GIN', 'referencia_tratada', 'Referencia_4', 
    'Setor', 'Valor', 'Área', 'Moeda', 'Operacao', 'Origem', 'Agente', 'Data_da_Pendência', 
    'Dias', 'Justificativa', 'status'
]


In [8]:
# Filtrar as linhas onde a coluna "Justificativa" inicia com "DashPerdas"
if 'Justificativa' in df.columns:
    df = df[df['Justificativa'].str.startswith('DashPerdas', na=False)]

In [9]:
# Incluir nova coluna "referencia_tratada" e aplicar a regra na coluna "Referencia_1"
if 'Referencia_1' in df.columns:
    df['referencia_tratada'] = df['Referencia_1'].apply(
        lambda x: x[4:-3] if isinstance(x, str) and x[:2] in ['FV', 'IM', 'VF', 'IF'] else x
    )
else:
    print("A coluna 'Referencia_1' não existe no DataFrame.")
    exit()

In [10]:
# Verificar se todas as colunas da nova ordem existem no DataFrame
colunas_existentes = [col for col in new_order if col in df.columns]

# Reorganizar as colunas
df = df.reindex(columns=colunas_existentes)

In [11]:
# # Verificar a existência das colunas necessárias para o cruzamento
# if 'referencia_tratada' in df.columns and 'NumeroBacen' in df_cambio.columns:
#     # Converter as colunas para o mesmo tipo (string)
#     df['referencia_tratada'] = df['referencia_tratada'].astype(str)
    
#     # Adicionar novas colunas ao DataFrame principal
#     novas_colunas = ["Recuperado_(S/N)", "Valor_recuperado", "Contabilizado_em_perdas_(S/N)", "Valor_de_perda"]
#     for coluna in novas_colunas:
#         df[coluna] = None

In [12]:
# Definir o caminho do arquivo CSV
caminho_csv = os.path.join(diretorio_cambio, nome_arquivo3)

# Verificar se o arquivo CSV existe
if os.path.exists(caminho_csv):
    try:
        # Carregar o arquivo CSV em um DataFrame, especificando o delimitador correto
        df_cambio = pd.read_csv(caminho_csv, sep=';', encoding='utf-8-sig')
        print("Arquivo CSV carregado com sucesso.")

        # Normatizar todas as colunas removendo espaços em branco
        df_cambio.columns = df_cambio.columns.str.strip()
        
        # Reduzir as colunas para as necessárias
        colunas_necessarias = [
            'GrpTpPessoa', 'Segmento', 'SegmentoGrupo', 
            'NomeBeneficiarioExterior', 'NomeClienteCustodiante', 
            'NumeroBacen', 'NumeroPreContrato'
        ]
        df_cambio = df_cambio[colunas_necessarias]
    except Exception as e:
        print(f"Erro ao carregar o arquivo CSV: {e}")
        exit()
else:
    print("Arquivo CSV não encontrado.")
    exit()

Arquivo CSV carregado com sucesso.


In [13]:
# Verificar a existência das colunas necessárias para o cruzamento
if 'referencia_tratada' in df.columns and 'NumeroBacen' in df_cambio.columns:
    # Converter as colunas para o mesmo tipo (string)
    df['referencia_tratada'] = df['referencia_tratada'].astype(str)
    
    # Adicionar novas colunas ao DataFrame principal
    novas_colunas = ["Recuperado_(S/N)", "Valor_recuperado", "Contabilizado_em_perdas_(S/N)", "Valor_de_perda","Colaborador_Edição", "Colaborador_Edição_Disciplina", "Colaborador_Legitimação" , "Colaborador_Legitimação_Disciplina"]
    for coluna in novas_colunas:
        df[coluna] = None

In [14]:
df.columns

Index(['Referencia_1', 'Referencia_2', 'GIN', 'referencia_tratada',
       'Referencia_4', 'Setor', 'Valor', 'Área', 'Moeda', 'Operacao', 'Origem',
       'Agente', 'Data_da_Pendência', 'Dias', 'Justificativa',
       'Recuperado_(S/N)', 'Valor_recuperado', 'Contabilizado_em_perdas_(S/N)',
       'Valor_de_perda', 'Colaborador_Edição', 'Colaborador_Edição_Disciplina',
       'Colaborador_Legitimação', 'Colaborador_Legitimação_Disciplina'],
      dtype='object')

In [15]:
#Tratamento da coluna NumeroBacen
df_cambio['NumeroBacen']=df_cambio['NumeroBacen'].astype(int)
df['Referencia_4']=df['Referencia_4'].astype(str).str.replace('.','')

In [16]:
# Converter as colunas para o mesmo tipo (string, por exemplo)
df['referencia_tratada'] = df['referencia_tratada'].astype(str)
df_cambio['NumeroBacen'] = df_cambio['NumeroBacen'].astype(str)

# Verificar se os valores de df['referencia_tratada'] estão em df_cambio['NumeroBacen']
df['status'] = df['referencia_tratada'].apply(lambda x: 'Ok' if x in df_cambio['NumeroBacen'].values else 'NOK')

# Realizar o merge para adicionar as colunas do df_cambio
df = df.merge(df_cambio[['NumeroBacen', 'GrpTpPessoa', 'Segmento', 'SegmentoGrupo', 
                                'NomeBeneficiarioExterior', 'NomeClienteCustodiante', 
                                ]], 
                     left_on='referencia_tratada', right_on='NumeroBacen', how='left')



In [17]:
# Definir o diretório e o nome do arquivo de saída
diretorio_saida = r'C:\\POWERBI\\Perdas\\Validação'
nome_arquivo_saida = 'Valid.xlsx'
caminho_arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

# Criar o diretório se não existir
os.makedirs(diretorio_saida, exist_ok=True)

# Salvar o DataFrame em um arquivo Excel
df.to_excel(caminho_arquivo_saida, index=False)

print(f"Arquivo salvo em: {caminho_arquivo_saida}")

Arquivo salvo em: C:\\POWERBI\\Perdas\\Validação\Valid.xlsx


In [18]:


df = pd.read_excel(rf"C:\\POWERBI\\Perdas\\Validação\Valid.xlsx")
#df = pd.read_excel(rf"C:\Users\t788066\Downloads\Valid.xlsx")

# Função para separar justificativas por data (com formatos dd/mm/yy e dd/mm)
def separar_por_data(row):
    justificativa = row['Justificativa']
    datas = re.findall(r'\d{2}/\d{2}(?:/\d{2})?', justificativa)
    partes = re.split(r'\d{2}/\d{2}(?:/\d{2})?', justificativa)
    
    novas_linhas = []

    # Adicionar a primeira parte que não tem data
    if partes[0].strip():
        novas_linhas.append((row['referencia_tratada'], partes[0].strip()))

    # Adicionar as partes que têm data
    for i, data in enumerate(datas):
        nova_justificativa = data + ' – ' + partes[i+1].strip()
        novas_linhas.append((row['referencia_tratada'], nova_justificativa))
        
    return novas_linhas

# Aplicar a função ao dataframe
novas_linhas = df.apply(separar_por_data, axis=1).explode().tolist()

# Criar um novo dataframe com as linhas separadas
novo_df = pd.DataFrame(novas_linhas, columns=['referencia_tratada', 'Justificativa'])

print(novo_df)

     referencia_tratada                                      Justificativa
0             444056654  DashPerdas - JABIL DO BRASIL INDUSTRIA ELETROE...
1             444056654  10/01/25 – - Solicitado devolução junto ao ban...
2             444056654  13/01/25 – – E-mail ao cliente pedindo auxílio...
3             444056654  14/01/25 – - Cliente informa que irá contatar ...
4             444056654  28/01/25 – - E-mail ao cliente questionando se...
5             444056654  06/02/25 – - Questionamos cliente quanto conta...
6             444056654  14/02/25 – - Houve fechamento a este beneficiá...
7             444056654  24/02/25 – - Encaminhamos e-mail ao cliente pe...
8             434698127  DashPerdas - RC9 COMERCIO E IMPORTACAO EIRELI ...
9             434698127   16/09 – - Solicitado devolução junto a investig.
10            434698127  17/10 – - E-mail para investig. para verificar...
11            434698127  23/10 – - E-mail ao cliente  solicitando auxíl...
12            434698127  

### Definir o diretório e o nome do arquivo de saída

In [19]:
# Caminho para a pasta base especificada
caminho_base = r"C:\\POWERBI\\Perdas\\Validação\Valid.xlsx"
caminho_historico = rf'\\Fscorppawbr18\orca_dados\Automação_Perdas\Base\Historico_Perdas.xlsx'



# Cria um DataFrame com os nomes dos arquivos
df = pd.read_excel(caminho_base)
# Lê o DataFrame histórico
df_historico = pd.read_excel(caminho_historico, dtype=str)

df = df.astype(str)
df_historico = df_historico.astype(str)

df_novos_dados = df[~df['GIN'].isin(df_historico['GIN'])]

# Adiciona os novos dados ao DataFrame histórico
df_historico_atualizado = pd.concat([df_historico, df_novos_dados], ignore_index=True)

# Exibe os novos dados que foram adicionados ao histórico
print("Total de casos na historico: "+ str(df_historico.shape[0]))
print("Total de casos novos: "+ str(df_novos_dados.shape[0]))
print("Total de casos na historico depois da add novos casos: "+ str(df_historico_atualizado.shape[0]))

# Salvando o DataFrame com os novos dados para ser enviado ao SharePoint
df_novos_dados.to_excel(caminho_base, index=False)

# Salva o DataFrame histórico atualizado
df_historico_atualizado.to_excel(caminho_historico, index=False)

Total de casos na historico: 14
Total de casos novos: 4
Total de casos na historico depois da add novos casos: 18


In [20]:
# Carregar o DataFrame original
df = pd.read_excel(rf"C:\\POWERBI\\Perdas\\Validação\Valid.xlsx")

# Função para separar justificativas por data (com formatos dd/mm/yy e dd/mm)
def separar_por_data(row):
    justificativa = row['Justificativa']
    datas = re.findall(r'\d{2}/\d{2}(?:/\d{2})?', justificativa)
    partes = re.split(r'\d{2}/\d{2}(?:/\d{2})?', justificativa)
    
    novas_linhas = []

    # Adicionar a primeira parte que não tem data
    if partes[0].strip():
        novas_linhas.append((row['referencia_tratada'], partes[0].strip()))

    # Adicionar as partes que têm data
    for i, data in enumerate(datas):
        nova_justificativa = data + ' – ' + partes[i+1].strip()
        novas_linhas.append((row['referencia_tratada'], nova_justificativa))
        
    return novas_linhas

# Aplicar a função ao dataframe
novas_linhas = df.apply(separar_por_data, axis=1).explode().tolist()

# Criar um novo dataframe com as linhas separadas
novo_df = pd.DataFrame(novas_linhas, columns=['referencia_tratada', 'Justificativa'])

# Substituir as colunas no DataFrame original
df['referencia_tratada'] = novo_df['referencia_tratada']
df['Justificativa'] = novo_df['Justificativa']

# Definir o diretório e o nome do arquivo de saída
diretorio_saida = rf'C:\\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Pendencias'
nome_arquivo_saida = 'Valid.xlsx'
caminho_arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

# Criar o diretório se não existir
os.makedirs(diretorio_saida, exist_ok=True)

# Salvar o DataFrame em um arquivo Excel
df.to_excel(caminho_arquivo_saida, index=False)

print(f"Arquivo salvo em: {caminho_arquivo_saida}")

Arquivo salvo em: C:\\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Pendencias\Valid.xlsx


In [21]:
# # Definir o diretório e o nome do arquivo de saída
# diretorio_saida = rf'C:\\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Pendencias'
# nome_arquivo_saida = 'Valid.xlsx'
# caminho_arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

# # Criar o diretório se não existir
# os.makedirs(diretorio_saida, exist_ok=True)

# # Salvar o DataFrame em um arquivo Excel
# df.to_excel(caminho_arquivo_saida, index=False)

# print(f"Arquivo salvo em: {caminho_arquivo_saida}")

In [22]:
# import openpyxl
# from openpyxl.worksheet.table import Table, TableStyleInfo
 
# # Caminho do arquivo Excel
# file_path = rf'C:\\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Pendencias'
 
 
# # Carregar o arquivo Excel
# wb = openpyxl.load_workbook(file_path)
 
# # Selecionar a planilha desejada (por exemplo, a primeira planilha)
# ws = wb.active
 
# # Definir o intervalo de dados (por exemplo, de A1 até a última célula preenchida)
# # Supondo que os dados começam na célula A1 e a primeira linha contém os cabeçalhos
# min_col = ws.min_column
# min_row = ws.min_row
# max_col = ws.max_column
# max_row = ws.max_row
 
# # Definir o intervalo de dados
# data_range = f"A{min_row}:{openpyxl.utils.get_column_letter(max_col)}{max_row}"
 
# # Criar a tabela
# table = Table(displayName="Table1", ref=data_range)
 
# # Adicionar estilo à tabela
# style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
#                        showLastColumn=False, showRowStripes=True, showColumnStripes=True)
# table.tableStyleInfo = style
 
# # Adicionar a tabela à planilha
# ws.add_table(table)
 
# # Salvar o arquivo Excel
# wb.save(file_path)

### Salvar em tabela sharepoint 

In [23]:

import openpyxl
from openpyxl.worksheet.table import Table, TableStyleInfo
 
# Caminho do arquivo Excel
file_path = r'C:\\Users\\T694816\\Santander Office 365\\IO-Controle Perdas e Pendencias - Documentos\\Pendencias\\Valid.xlsx'
 
 
# Carregar o arquivo Excel
wb = openpyxl.load_workbook(file_path)
 
# Selecionar a planilha desejada (por exemplo, a primeira planilha)
ws = wb.active

display(ws)
 
# Definir o intervalo de dados (por exemplo, de A1 até a última célula preenchida)
# Supondo que os dados começam na célula A1 e a primeira linha contém os cabeçalhos
min_col = ws.min_column
min_row = ws.min_row
max_col = ws.max_column
max_row = ws.max_row
 
# Definir o intervalo de dados
data_range = f"A{min_row}:{openpyxl.utils.get_column_letter(max_col)}{max_row}"
 
# Criar a tabela
table = Table(displayName="Table1", ref=data_range)
 
# Adicionar estilo à tabela
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
table.tableStyleInfo = style
 
# Adicionar a tabela à planilha
ws.add_table(table)
 
# Salvar o arquivo Excel
wb.save(file_path)

<Worksheet "Sheet1">

In [24]:
# # Definir o diretório e o nome do arquivo de saída
# diretorio_saida = r'C:\\POWERBI\\Perdas\\Validação'
# nome_arquivo_saida = 'Valid.xlsx'
# caminho_arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

# # Criar o diretório se não existir
# os.makedirs(diretorio_saida, exist_ok=True)

# # Salvar o DataFrame em um arquivo Excel
# df.to_excel(caminho_arquivo_saida, index=False)

# print(f"Arquivo salvo em: {caminho_arquivo_saida}")

In [25]:
# Definir o diretório e o nome do arquivo
diretorio = r'C:\\POWERBI\\Perdas\\Bases'
nome_arquivo2 = 'P16-Back_3_202412.csv'
caminho_arquivo2 = os.path.join(diretorio, nome_arquivo2)

# Verificar se o arquivo CSV existe
if os.path.exists(caminho_arquivo2):
    # Carregar o arquivo CSV em um DataFrame sem cabeçalho
    df_back = pd.read_csv(caminho_arquivo2, header=None, sep=';')
    print("Arquivo CSV carregado com sucesso.")
    
    # Selecionar as colunas de índice 2, 7, 9, 12 e 22
    colunas_indices = [2, 5, 9, 11, 22]
    colunas_nomes = [
        'Codigo_Evento_Area', 'Tipo_de_Dado_Financeiro', 'Data_contabilização', 
        'Valor_do_lançamento', 'Descrição_do_evento'
    ]

    df_back = df_back.iloc[:, colunas_indices]
    df_back.columns = colunas_nomes  # Renomear as colunas
    
    # Função para extrair o comentário específico
    def extract_comment(text):
        match = re.search(r'(Pagamento efetuado a beneficiário incorreto.*?)(?=\d{2}/\d{2}/\d{2})', text)
        return match.group(1).strip() if match else None

    # Aplicar a função para criar a nova coluna de comentário
    df_back['Comentario'] = df_back['Descrição_do_evento'].apply(extract_comment)
    

Arquivo CSV carregado com sucesso.


In [26]:
# # Definir o diretório e o nome do arquivo
# diretorio = r'C:\\POWERBI\\Perdas\\Bases'
# nome_arquivo2 = 'P16-Back_3_202412.csv'
# caminho_arquivo2 = os.path.join(diretorio, nome_arquivo2)

# # Verificar se o arquivo CSV existe
# if os.path.exists(caminho_arquivo2):
#     # Carregar o arquivo CSV em um DataFrame sem cabeçalho
#     df_back = pd.read_csv(caminho_arquivo2, header=None, sep=';')
#     print("Arquivo CSV carregado com sucesso.")
    
#     # Selecionar as colunas de índice 2, 6, 10, 12 e 23
#     colunas_indices = [2, 7, 9, 12, 22]
#     colunas_nomes = [
#         'Codigo_Evento_Area', 'Tipo_de_Dado_Financeiro', 'Data_contabilização', 
#         'Valor_do_lançamento', 'Descrição_do_evento'
#     ]

#     df_back = df_back.iloc[:, colunas_indices]
#     df_back.columns = colunas_nomes  # Renomear as colunas
    
# #     # Exibir as primeiras linhas do DataFrame reduzido
# #     print("Primeiras linhas do DataFrame reduzido:")
# #     print(df_back.head())
# # else:
# #     print("Arquivo CSV não encontrado.")

In [27]:
# Adicionar novas colunas ao DataFrame
novas_colunas = [
    "Sistema", "Colaborador_edição", "Medida_disciplinar_edição", "Colaborador_da_legitimação", "Medida_disciplinar_legit",
    "Chances_de_recuperar_o_valor", "Status_da_Recuperação", "Observações", "Previsão_de_regularização","GIN"
]
for coluna in novas_colunas:
    df_back[coluna] = None

# # Exibir os nomes das colunas após adicionar novas colunas
# print("Nomes das colunas após adicionar novas colunas:")
# print(df_back.columns)

In [28]:
# Converter as colunas para o mesmo tipo (string, por exemplo)
df_back['Codigo_Evento_Area'] = df_back['Codigo_Evento_Area'].astype(str)
df_cambio['NumeroBacen'] = df_cambio['NumeroBacen'].astype(str)

# Verificar se os valores de df_back['Codigo_Evento_Area'] estão em df_cambio['NumeroBacen']
df_back['status'] = df_back['Codigo_Evento_Area'].apply(lambda x: 'Ok' if x in df_cambio['NumeroBacen'].values else 'NOK')

# Realizar o merge para adicionar as colunas do df_cambio para os casos 'Ok'
df_back_ok = df_back[df_back['status'] == 'Ok']
df_back_ok = df_back_ok.merge(df_cambio[['NumeroBacen', 'GrpTpPessoa', 'Segmento', 'SegmentoGrupo', 
                                         'NomeBeneficiarioExterior', 'NomeClienteCustodiante', 
                                         ]], 
                              left_on='Codigo_Evento_Area', right_on='NumeroBacen', how='left')


In [29]:
# Caminho para a pasta base especificada
caminho_base = r"C:\\POWERBI\\Perdas\\Validação\Valid.xlsx"
caminho_historico = rf'\\Fscorppawbr18\orca_dados\Automação_Perdas\Base\Historico_IRO.xlsx'



# Cria um DataFrame com os nomes dos arquivos
df = pd.read_excel(caminho_base)
# Lê o DataFrame histórico
df_historico = pd.read_excel(caminho_historico, dtype=str)

df = df.astype(str)
df_historico = df_historico.astype(str)

df_novos_dados = df[~df['NumeroBacen'].isin(df_historico['NumeroBacen'])]

# Adiciona os novos dados ao DataFrame histórico
df_historico_atualizado = pd.concat([df_historico, df_novos_dados], ignore_index=True)

# Exibe os novos dados que foram adicionados ao histórico
print("Total de casos na historico: "+ str(df_historico.shape[0]))
print("Total de casos novos: "+ str(df_novos_dados.shape[0]))
print("Total de casos na historico depois da add novos casos: "+ str(df_historico_atualizado.shape[0]))

# Salvando o DataFrame com os novos dados para ser enviado ao SharePoint
df_novos_dados.to_excel(caminho_base, index=False)

# Salva o DataFrame histórico atualizado
df_historico_atualizado.to_excel(caminho_historico, index=False)

Total de casos na historico: 189
Total de casos novos: 4
Total de casos na historico depois da add novos casos: 193


### Definir o diretório e o nome do arquivo de saída no C:

In [30]:
# Definir o diretório e o nome do arquivo de saída
diretorio_saida = r'C:\\POWERBI\\Perdas\\Validação'
nome_arquivo_saida = 'IRO.xlsx'
caminho_arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

# Criar o diretório se não existir
os.makedirs(diretorio_saida, exist_ok=True)

# Salvar o DataFrame em um arquivo Excel
df_back_ok.to_excel(caminho_arquivo_saida, index=False)

print(f"Arquivo salvo em: {caminho_arquivo_saida}")

Arquivo salvo em: C:\\POWERBI\\Perdas\\Validação\IRO.xlsx


In [31]:
# Definir o diretório e o nome do arquivo de saída
diretorio_saida = rf'C:\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Perdas'
nome_arquivo_saida = 'IRO.xlsx'
caminho_arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

# Criar o diretório se não existir
os.makedirs(diretorio_saida, exist_ok=True)

# Salvar o DataFrame em um arquivo Excel
df_back_ok.to_excel(caminho_arquivo_saida, index=False)

print(f"Arquivo salvo em: {caminho_arquivo_saida}")

Arquivo salvo em: C:\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Perdas\IRO.xlsx


### Salvar em tabela sharepoint 

In [32]:
import openpyxl
from openpyxl.worksheet.table import Table, TableStyleInfo
 
# Caminho do arquivo Excel
file_path = r'C:\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Perdas\\IRO.xlsx'
 
 
# Carregar o arquivo Excel
wb = openpyxl.load_workbook(file_path)
 
# Selecionar a planilha desejada (por exemplo, a primeira planilha)
ws = wb.active

display(ws)
 
# Definir o intervalo de dados (por exemplo, de A1 até a última célula preenchida)
# Supondo que os dados começam na célula A1 e a primeira linha contém os cabeçalhos
min_col = ws.min_column
min_row = ws.min_row
max_col = ws.max_column
max_row = ws.max_row
 
# Definir o intervalo de dados
data_range = f"A{min_row}:{openpyxl.utils.get_column_letter(max_col)}{max_row}"
 
# Criar a tabela
table = Table(displayName="Table1", ref=data_range)
 
# Adicionar estilo à tabela
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
table.tableStyleInfo = style
 
# Adicionar a tabela à planilha
ws.add_table(table)
 
# Salvar o arquivo Excel
wb.save(file_path)

<Worksheet "Sheet1">

### # Definir o diretório e o nome do arquivo de saída

In [33]:
# # Definir o diretório e o nome do arquivo de saída
# diretorio_saida = rf'C:\Users\T694816\Santander Office 365\IO-Controle Perdas e Pendencias - Documentos\Perdas'
# nome_arquivo_saida = 'IRO.xlsx'
# caminho_arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

# # Criar o diretório se não existir
# os.makedirs(diretorio_saida, exist_ok=True)

# # Salvar o DataFrame em um arquivo Excel
# df_back_ok.to_excel(caminho_arquivo_saida, index=False)

# print(f"Arquivo salvo em: {caminho_arquivo_saida}")